# Option Pricing with Binomial Model

In [1]:
import numpy as np
from math import comb
import math

should be answered by building a 15-period binomial model whose parameters should be calibrated to a Black-Scholes geometric Brownian motion model with: T= .25, S_0 = 100, r = 2%, \sigma = 30%, dividend yield of c = 1%.

Binomial model should have u = 1.0395

Define constants that are given

In [2]:
T = .25
S0 = 100
r = 0.02
sigma = .3
c = .01
n = 15 # of periods

Calibrate binomial model parameter using Black-Scholes

In [3]:
dt = T/n   # the size of each period

In [4]:
Rn = np.exp(r*dt)                          # the estimated interest rate in an n period model
un = np.exp(sigma*dt**0.5)                 # up move per period
dn = 1/un                                  # down move per period
qn = (np.exp((r - c)*dt) - dn) / (un - dn) # risk neutral probability

### Q1

In [5]:
K = 110
C0 = (1/Rn**n) * sum([comb(n, j)*qn**j*(1-qn)**(n-j)*max(S0*un**j*dn**(n-j) - K, 0) for j in range(n)])

In [6]:
print(f"The value of a european call option is ${C0:.2f} (apparently the right answer)")

The value of a european call option is $2.60 (apparently the right answer)


In [7]:
def reduce_call(values, prices, n=15, american=True, debug=False):
    # values is a list of list with option values at each time period
    results = values[0]
    depth = n - len(values)
    if debug:
        print(f"len: {len(prices)}\tdepth: {depth}")

    if american:
        prev_prices = prices[depth]

    early = False

    # the last layer of the binomial lattice and takes a step back
    new_results = []
    for i in range(len(results) - 1):
        # get the value of the option if continues
        risk_neutral_values = (qn*results[i] + (1-qn)*results[i + 1])/Rn
        if debug:
            print(f"RNV: {risk_neutral_values:.2f}\tRes{i}: {results[i]:.2f}\tRes{i+1}: {results[i+1]:.2f}")
            if american: print(f"Prev price: {prev_prices[i]:.2f}\tearly exec gain: {prev_prices[i] - K:.2f}")
        # the risk neutral value, i.e. executing the options or continuing
        if american:
            val = max(max(prev_prices[i] - K,0), risk_neutral_values)
            if val == prev_prices[i] - K:
                early = True
        else:
            val = max(0, risk_neutral_values)
        new_results.append(val)
        
    values.insert(0, new_results)
    return values, early

In [8]:
def build_lattice():
    lattice = []
    
    # how far down the lattice we are
    for k in range(n + 1):
        subprices = []

        # generate all the vertical values
        for i in range(k + 1):
            subprice = S0*un**(k-i)*dn**(i)
            subprices.append(subprice)
            
        lattice.append(subprices)

    return lattice

In [9]:
lattice = build_lattice()

In [10]:
# generate the final prices that will be at the end of the lattice
P_final = []

for i in range(n + 1):
    subprice = S0*un**(n-i)*dn**(i) - K # should be expiry value
    P_final.append(max(subprice, 0))
P_final = [P_final]

for i in range(n):
    P_final, _ = reduce_call(P_final, lattice)
call_price = P_final[0][0]

In [11]:
print(f"Value of the american call option is ${call_price:.2f}")
Q1 = round(call_price, 2)

Value of the american call option is $2.60


### Q2

Compute the price of an American put option with strike K=110 and maturity T=0.25

In [12]:
def reduce_put(values, prices, american=True):
    early_exec = False
    
    # values is a list of list with option values at each time period
    results = values[0]
    depth = n - len(values)
    prices = lattice[depth]
    prev_prices = lattice[depth]
    
    # the last layer of the binomial lattice and takes a step back
    new_results = []
    for i in range(len(results) - 1):
        # get the value of the option if continues
        risk_neutral_values = (qn*results[i] + (1-qn)*results[i + 1])/Rn

        # the risk neutral value, i.e. executing the options or continuing
        if american:
            val = max(max(K - prev_prices[i], 0), risk_neutral_values)
            if val == K - prev_prices[i]:
                early_exec = True
        else:
            val = max(0, risk_neutral_values)
        new_results.append(val)
        
    values.insert(0, new_results)
    return values, early_exec

In [13]:
# generate the final prices that will be at the end of the lattice
P_final = []

for i in range(n + 1):
    subprice = K - S0*un**(n-i)*dn**(i) # should be expiry value
    P_final.append(max(subprice, 0))
P_final = [P_final]

early = False
exercise_period = n

for i in range(n):
    P_final, early_exec = reduce_put(P_final, lattice)
    if early_exec:
        early = True
        exercise_period = n - 1 - i   # since we are guranteed to be in a lower period

put_price = P_final[0][0]

In [14]:
Q2 = round(put_price, 2)
print(f"the put price is ${put_price:.2f}")

the put price is $12.36


# Q3

In [15]:
Q3 = 'should' if early else 'should not'
print(f"You {'should' if early else 'should not'} early exercise")

You should early exercise


# Q4

In [16]:
Q4 = exercise_period
print(f"Earliest exercise period is period {exercise_period}")

Earliest exercise period is period 5


# Q5

Do the options satisfy the put-call parity

In [17]:
left = put_price + S0*np.exp(-1*c*T)
right = call_price + K*np.exp(-1*r*T)

In [18]:
print(f'left: {left} -vs- right: {right}')
Q5 = "Yes" if left == right else "No"

left: 112.11009703703091 -vs- right: 112.0554498441616


 No, put call parity is not satisfied.

# Q6

Compute the fair value of an american call with strike K=110 and maturaity n = 10. Writen on future that expires in 15 periods.

Creat the futures price lattice

In [19]:
P_final = []
for i in range(n + 1):
    subprice = S0*un**(n-i)*dn**(i) # should be expiry value
    P_final.append(max(subprice, 0))
P_final = [P_final]

In [20]:
def reduce_futures(prices):
    results = prices[0]
    
    # the last layer of the binomial lattice and takes a step back
    new_results = []
    for i in range(len(results) - 1):
        # get the value of the option if continues
        risk_neutral_values = (qn*results[i] + (1-qn)*results[i + 1])    # why no "/R"?
        new_results.append(risk_neutral_values)
        
    prices.insert(0, new_results)
    return prices

In [21]:
for i in range(n):
    P_final = reduce_futures(P_final)

In [22]:
futures_lattice = P_final[:11]

In [23]:
call_lattice = [[max(x - K, 0) for x in futures_lattice[-1]]]
earliest = 10
for i in range(10):
    call_lattice, early = reduce_call(call_lattice, futures_lattice, n=10)
    if early:
        earliest = 9 - i


In [24]:
Q6 = round(call_lattice[0][0], 2)
print(f"The price of the call on the future is ${Q6}")

The price of the call on the future is $1.66


# Q7

What is the earliest time period in which you might want to exercise the American futures option of Q6?

In [25]:
Q7 = earliest
print(f'Might want to exercise at period: {Q7}')

Might want to exercise at period: 7


# Q8

Compute the value of a chooser option which expires after n = 10. At expiry owner chooses between a european call or put. call and put K = 100, n = 15

In [38]:
K = 100
# set for 10 periods
n=15
dt = T/n   # the size of each period
Rn = np.exp(r*dt)                          # the estimated interest rate in an n period model
un = np.exp(sigma*dt**0.5)                 # up move per period
dn = 1/un                                  # down move per period
qn = (np.exp((r - c)*dt) - dn) / (un - dn) # risk neutral probability

# generate the final prices that will be at the end of the lattice
call_exec_value = []
put_exec_value = []
for i in range(n + 1):
    subprice = S0*un**(n-i)*dn**(i) - K # should be expiry value
    put_subprice = K - S0*un**(n-i)*dn**(i)
    call_exec_value.append(max(subprice, 0))
    put_exec_value.append(max(put_subprice, 0))
    
put_exec_value = [put_exec_value]
call_exec_value = [call_exec_value]

In [39]:
n = 15
for i in range(n):
    call_exec_value, _ = reduce_call(call_exec_value, [], american=False, n=15)


for i in range(n):
    put_exec_value, _ = reduce_put(put_exec_value, [], american=False)
    

In [44]:
# set for 10 periods
n=10
# dt = T/n   # the size of each period
# Rn = np.exp(r*dt)                          # the estimated interest rate in an n period model
# un = np.exp(sigma*dt**0.5)                 # up move per period
# dn = 1/un                                  # down move per period
# qn = (np.exp((r - c)*dt) - dn) / (un - dn) # risk neutral probability

last_call = call_exec_value[n]
last_put = put_exec_value[n]

prices = [[max(x, y) for x, y in zip(last_call, last_put)]]

In [45]:
print(prices)

for i in range(n):
    prices, _ = reduce_call(prices, [], n=10, american=False)

[[47.34341646977959, 36.37349148073286, 26.221216976064902, 16.953365416112103, 9.118798112983885, 3.6667757928995517, 8.308878288623301, 14.370587611723995, 20.634933129113413, 26.538047676675575, 32.00116974717185]]


In [46]:
Q8 = round(prices[0][0], 2)
print(f'Price of chooser is ${Q8}')

Price of chooser is $10.8


In [47]:
print(f"Q1\t{Q1}")
print(f"Q2\t{Q2}")
print(f"Q3\t{Q3}")
print(f"Q4\t{Q4}")
print(f"Q5\t{Q5}")
print(f"Q6\t{Q6}")
print(f"Q7\t{Q7}")
print(f"Q8\t${Q8}")

Q1	2.6
Q2	12.36
Q3	should
Q4	5
Q5	No
Q6	1.66
Q7	7
Q8	$10.8
